<a href="https://colab.research.google.com/github/hmyrcmn/bitirme/blob/main/UploadModelToHuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch numpy


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [2]:
from google.colab import files
import json


# Yüklenen dosyaları okuyun
cv_file = "/content/cv.json"
startup_file = "/content/Startup.json"

with open(cv_file, 'r') as f:
    cv_data = json.load(f)

with open(startup_file, 'r') as f:
    startup_data = json.load(f)


In [3]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

# BERT modelini ve tokenizer'ını yükleyin
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

 get_embedding fonksiyonu, BERT modelinin sadece son gizli katmanının çıkışını kullanarak özellik çıkarır. Bu durumda, modelin en son katmanından elde edilen çıkışlar kullanılır.

# MODEL

get_embedding_from_layer fonksiyonu, BERT modelinin tüm giriş katmanlarını kullanarak özellik çıkarır. layer_num parametresi, hangi katmanın kullanılacağını belirler

In [4]:
import torch
from transformers import BertTokenizer, BertModel

# BERT modelini ve tokenizer'ını yükleyin
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Özellik çıkarma fonksiyonları
def get_embedding_from_layer(text, layer_num):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs, output_hidden_states=True)
    layer_output = outputs.hidden_states[layer_num]
    return torch.mean(layer_output, dim=1).detach().numpy()

def get_text_from_cv(cv):
    competence = ' '.join(cv.get('competence', []))
    sectors = ' '.join(cv.get('CV Experience & Know How', {}).get('Sectors', []))
    experiences = ' '.join(exp.get('description', '') for exp in cv.get('CV Experience & Know How', {}).get('Experience', []))
    return f"{competence} {sectors} {experiences}"

def get_text_from_startup(startup):
    return f"{startup.get('tags', '')}"

# CV ve startup verilerini gömülü vektörlere dönüştürün
cv_embeddings = [get_embedding_from_layer(get_text_from_cv(cv), layer_num=11) for cv in cv_data] # Örnek olarak 11. katmanı kullanıyoruz
startup_embeddings = [get_embedding_from_layer(get_text_from_startup(startup), layer_num=11) for startup in startup_data]

# Benzerlik hesaplama ve en uygun CV'yi bulma fonksiyonları
def cosine_similarity(vec1, vec2):
    vec1 = vec1.flatten()
    vec2 = vec2.flatten()
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def find_best_cv_for_startup(startup_index):
    startup_embedding = startup_embeddings[startup_index]
    similarities = [cosine_similarity(startup_embedding, cv_embedding) for cv_embedding in cv_embeddings]
    best_cv_index = np.argmax(similarities)
    return cv_data[best_cv_index], similarities[best_cv_index]

# Sonuçları görüntüleyin
for i, startup in enumerate(startup_data):
    best_cv, score = find_best_cv_for_startup(i)
    print(f"Startup: {startup['business_name']}")
    print(f"En uygun CV: {best_cv['name']} (Skor: {score:.2f})")
    print()


Startup: PlayerGo
En uygun CV: Selin Öztürk (Skor: 0.69)

Startup: CMO Index – Fractional CMO Directory
En uygun CV: Rıza Doğan (Skor: 0.80)

Startup: Inqud
En uygun CV: Tolga Alkan (Skor: 0.78)

Startup: TaxInsight
En uygun CV: Jenny Bartsch (Skor: 0.71)

Startup: LogoYa
En uygun CV: Adrian Thoma (Skor: 0.80)

Startup: Riddep
En uygun CV: Jale Türk (Skor: 0.81)

Startup: Linkhouse
En uygun CV: Bengü İnan (Skor: 0.86)

Startup: Español Chévere
En uygun CV: Bengü Deniz (Skor: 0.78)

Startup: ChatWithCloud
En uygun CV: Alexandra Rudl (Skor: 0.80)

Startup: Health Folder
En uygun CV: Dilek Yıldız (Skor: 0.81)

Startup: Truck1.sg
En uygun CV: Günter Meier (Skor: 0.80)

Startup: 3CHEAPS
En uygun CV: Ulrich Benz (Skor: 0.77)

Startup: RENEWABLE ENERGY HOLDING SA
En uygun CV: Ali Demir (Skor: 0.79)

Startup: RenderLion
En uygun CV: Alexandra Rudl (Skor: 0.77)

Startup: Planet Wild
En uygun CV: Can Yılmaz (Skor: 0.79)

Startup: Zapant
En uygun CV: Alexandra Rudl (Skor: 0.78)

Startup: 20ROBOTS

# convert token upload to hugigng face platform

In [20]:
!pip install transformers torch numpy huggingface_hub
from google.colab import files
import json
from huggingface_hub import notebook_login

# Hugging Face Hub'a giriş yapın
notebook_login()


# 2. Model  Tokenizer Save Hugging Face


In [21]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

# BERT modelini ve tokenizer'ını yükleyin
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Model ve tokenizer'ı kaydetme
model.save_pretrained("saved_model")
tokenizer.save_pretrained("saved_model")


('saved_model/tokenizer_config.json',
 'saved_model/special_tokens_map.json',
 'saved_model/vocab.txt',
 'saved_model/added_tokens.json')

In [26]:
from huggingface_hub import HfApi

# Hugging Face API Object creating
api = HfApi()


folder_path = "saved_model"
repo_id = "hmyrcmn/cvMentorMatch"

# Model klasörünü yükleyin
api.upload_folder(
    folder_path=folder_path,
    repo_id=repo_id
)


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hmyrcmn/cvMentorMatch/commit/a9427233de3a8a649ce155275c8ed024528ed17e', commit_message='Upload folder using huggingface_hub', commit_description='', oid='a9427233de3a8a649ce155275c8ed024528ed17e', pr_url=None, pr_revision=None, pr_num=None)

# Test AND Use UPLOADED TOKEN




In [29]:
from transformers import AutoTokenizer, AutoModel

# Modeli ve tokenizer'ı yükle
model_name = "hmyrcmn/cvMentorMatch"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Örnek metni tokenize et
text = "I have experience in entrepreneurship and I have developed artificial intelligence projects."
inputs = tokenizer(text, return_tensors="pt")

# Modeli kullanarak gömülü vektörü al
outputs = model(**inputs)
embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()

print(embeddings)


tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

[[ 2.09040701e-01  2.26869896e-01 -1.36581659e-01 -1.93200052e-01
   4.04356003e-01 -3.94908875e-01 -2.12636381e-01  5.15552580e-01
  -1.20204255e-01 -2.60190457e-01  1.25499919e-01 -4.10204738e-01
   1.42821148e-01  1.55766502e-01 -1.34272771e-02  4.25691791e-02
  -7.57537335e-02  1.23466939e-01 -1.98494390e-01  5.30699372e-01
  -1.69345781e-01  2.26340115e-01 -1.52649805e-01  8.30412090e-01
   3.54151547e-01  4.62100841e-02 -3.16637605e-01  2.88275033e-01
  -2.69729227e-01 -1.52565107e-01  3.36089820e-01  5.63694909e-02
  -1.65757418e-01  1.38241902e-01  2.86453068e-01 -2.93592989e-01
  -2.60978341e-01 -4.07557227e-02 -7.22693086e-01  2.70976871e-01
  -1.85847223e-01 -2.60803521e-01  5.26083913e-03  8.77429321e-02
  -2.00891122e-01 -1.63724363e-01 -4.48752791e-02 -2.09089890e-02
   3.93813044e-01  1.79656506e-01 -4.55660880e-01  2.36784503e-01
  -1.90027446e-01 -5.55885732e-01  3.80889922e-01  8.34937632e-01
  -5.86155578e-02 -5.15845001e-01 -1.10386483e-01 -9.87646952e-02
   3.32719

In [40]:
import json
import numpy as np
from transformers import BertModel, BertTokenizer

# CV'yi JSON formatından yükle
cv_json = """
{
  "name": "Hümeyra ÇİMEN",
  "contact": {
    "linkedin": "https://www.linkedin.com/in/hmyrcmn/",
    "email": "cimenhumeyra2001@gmail.com",
    "github": "https://github.com/hmyrcmn"
  },
  "education": [
    {
      "institution": "Bursa Technical University",
      "location": "Bursa, Türkiye",
      "degree": "English Preparatorg Program",
      "graduation_date": "2024",
      "description": "Successfully completed English Preparatorg Program. Scored 80 out of 100 in the BTU YDS exam."
    },
    {
      "institution": "Kayseri-Tomarza Science High School",
      "location": "Kayseri, Türkiye",
      "description": "GPA: 88/100"
    }
  ],
  "experience": [
    {
      "position": "Project Manager and Full Stack Developer",
      "company": "ASE Bilişim",
      "location": "Remote",
      "dates": "01.2021 – 10.2021",
      "responsibilities": [
        "Participated in a 3-month Technical Training Period with a 7-member team, gaining practical experience in project development.",
        "Collaborated within a team of 24 engineers, showcasing adaptability and teamwork skills."
      ]
    },
    {
      "position": "Ai Developer",
      "company": "SETINT AI Hospital",
      "location": "Istanbul, Türkiye",
      "dates": "02.2022 – 04.2022",
      "responsibilities": [
        "Contributed to a web back-end development team during a one-month internship.",
        "Conducted literature review, machine learning algorithms development, and research activities."
      ]
    },
    {
      "position": "Web Developer",
      "company": "BURULAS",
      "location": "Bursa, Türkiye",
      "dates": "08.2022 – 10.2022",
      "responsibilities": [
        "Collaborated with a team in the development of a mobile and web application.",
        "Worked on the parking automation system development team and on a team in the development of a mobile and web application."
      ]
    },
    {
      "position": "Unity Game Developer",
      "company": "ATF STUDIO",
      "location": "Ankara, Türkiye",
      "dates": "08.2023 – 10.2023",
      "responsibilities": [
        "Actively contributed to various game projects in both 2D and 3D scenes.",
        "Gained valuable hands-on experience by developing technical skills in Unity game development."
      ]
    },
    {
      "position": "Ai (Deep Learning – ML) Developer",
      "company": "Kul Elektronik",
      "location": "Bursa, Türkiye",
      "dates": "10.2023 - ...",
      "responsibilities": [
        "Developed projects on deep learning algorithms.",
        "Worked on the object detection project on images and AI-supported fault detection project using signals."
      ]
    }
  ],
  "leadership_and_activities": [
    {
      "organization": "Matro Teknofest Team",
      "location": "City, State",
      "dates": "Month Year – Month Year",
      "description": "As the President and Software Lead of Matro Submarine Technofest Team, led the software development for the underwater vehicle project, focusing on device control and motor control. Made significant contributions through effective communication and team coordination."
    },
    {
      "organization": "Global Ai Hub",
      "description": "Participated in and managed many bootcamps at the Global AI Hub. Teamed up with many people, managed bootcamps."
    }
  ],
  "skills_and_interests": {
    "technical": ["python", "c", "c#", "java", "SQL", "WEB Programming (Php, Html, Js, Mysql)"],
    "language": "English Professional Work Competence",
    "interests": "Passionate about exploring the latest developments, coding for personal projects, participating in hackathons, staying informed about industry trends. Also follow science fiction and technology-related films. Engaging in outdoor activities like hiking and photography reflects commitment to a balanced lifestyle."
  }
}
"""  # CV JSON formatındaki veriyi buraya yapıştırın

cv_data = json.loads(cv_json)

# BERT modeli ve tokenizer'ını yükle
model_name = "dbmdz/bert-base-turkish-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# CV metnini işle
cv_text = "\n".join([f"{key}: {value}" for key, value in cv_data.items()])
inputs = tokenizer(cv_text, return_tensors="pt", max_length=512, truncation=True)
outputs = model(**inputs)
cv_embedding = outputs.last_hidden_state.mean(dim=1).detach().numpy()


inputs = tokenizer(cv_text, return_tensors="pt", max_length=512, truncation=True)
outputs = model(**inputs)
cv_embedding = outputs.last_hidden_state.mean(dim=1).detach().numpy()

# Startup.json dosyasını yükle
startup_json = """
[
    {
        "title": "Poland",
        "business_name": "PlayerGo",
        "category": "Poland",
        "based_in": "Krakow",
        "tags": "games, hardware, controllers, gaming, videogames",
        "founded": "2010"
    },
    {
        "title": "Turkey",
        "business_name": "TechHub",
        "category": "Technology",
        "based_in": "Istanbul",
        "tags": "tech, startups, innovation, software, hardware",
        "founded": "2015"
    },
    {
        "title": "Germany",
        "business_name": "AutoCraft",
        "category": "Automotive",
        "based_in": "Berlin",
        "tags": "cars, vehicles, automotive, engineering",
        "founded": "2005"
    },{
    "title": "Unity Game Development",
    "business_name": "PixelForge Studios",
    "category": "Game Development",
    "based_in": "San Francisco",
    "tags": "unity, game development, virtual reality, augmented reality",
    "founded": "2010"
}

]
"""  # startup.json formatındaki veriyi buraya yapıştırın

startup_data = json.loads(startup_json)

# startup.json dosyasından şirket metinlerini al
startup_texts = [startup["title"] + " " + startup["business_name"] for startup in startup_data]

# BERT gömme (embedding) işlemlerini yap
startup_embeddings = []
for startup_text in startup_texts:
    inputs = tokenizer(startup_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model(**inputs)
    startup_embedding = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    startup_embeddings.append(startup_embedding)

# Kosinüs benzerliği hesaplama
similarities = [np.dot(cv_embedding, startup_embedding.T) / (np.linalg.norm(cv_embedding) * np.linalg.norm(startup_embedding)) for startup_embedding in startup_embeddings]

# En uygun startup'ı bul
best_match_index = np.argmax(similarities)
best_match_startup = startup_data[best_match_index]

print("En uygun startup:")
print(best_match_startup)


En uygun startup:
{'title': 'Unity Game Development', 'business_name': 'PixelForge Studios', 'category': 'Game Development', 'based_in': 'San Francisco', 'tags': 'unity, game development, virtual reality, augmented reality', 'founded': '2010'}


In [41]:
import json
import numpy as np
from transformers import BertModel, BertTokenizer

# CV'yi JSON formatından yükle
cv_json = """
{
  "name": "Hümeyra ÇİMEN",
  "contact": {
    "linkedin": "https://www.linkedin.com/in/hmyrcmn/",
    "email": "cimenhumeyra2001@gmail.com",
    "github": "https://github.com/hmyrcmn"
  },
  "education": [
    {
      "institution": "Bursa Technical University",
      "location": "Bursa, Türkiye",
      "degree": "English Preparatorg Program",
      "graduation_date": "2024",
      "description": "Successfully completed English Preparatorg Program. Scored 80 out of 100 in the BTU YDS exam."
    },
    {
      "institution": "Kayseri-Tomarza Science High School",
      "location": "Kayseri, Türkiye",
      "description": "GPA: 88/100"
    }
  ],
  "experience": [
    {
      "position": "Project Manager and Full Stack Developer",
      "company": "ASE Bilişim",
      "location": "Remote",
      "dates": "01.2021 – 10.2021",
      "responsibilities": [
        "Participated in a 3-month Technical Training Period with a 7-member team, gaining practical experience in project development.",
        "Collaborated within a team of 24 engineers, showcasing adaptability and teamwork skills."
      ]
    },
    {
      "position": "Ai Developer",
      "company": "SETINT AI Hospital",
      "location": "Istanbul, Türkiye",
      "dates": "02.2022 – 04.2022",
      "responsibilities": [
        "Contributed to a web back-end development team during a one-month internship.",
        "Conducted literature review, machine learning algorithms development, and research activities."
      ]
    },
    {
      "position": "Web Developer",
      "company": "BURULAS",
      "location": "Bursa, Türkiye",
      "dates": "08.2022 – 10.2022",
      "responsibilities": [
        "Collaborated with a team in the development of a mobile and web application.",
        "Worked on the parking automation system development team and on a team in the development of a mobile and web application."
      ]
    },
    {
      "position": "Unity Game Developer",
      "company": "ATF STUDIO",
      "location": "Ankara, Türkiye",
      "dates": "08.2023 – 10.2023",
      "responsibilities": [
        "Actively contributed to various game projects in both 2D and 3D scenes.",
        "Gained valuable hands-on experience by developing technical skills in Unity game development."
      ]
    },
    {
      "position": "Ai (Deep Learning – ML) Developer",
      "company": "Kul Elektronik",
      "location": "Bursa, Türkiye",
      "dates": "10.2023 - ...",
      "responsibilities": [
        "Developed projects on deep learning algorithms.",
        "Worked on the object detection project on images and AI-supported fault detection project using signals."
      ]
    }
  ],
  "leadership_and_activities": [
    {
      "organization": "Matro Teknofest Team",
      "location": "City, State",
      "dates": "Month Year – Month Year",
      "description": "As the President and Software Lead of Matro Submarine Technofest Team, led the software development for the underwater vehicle project, focusing on device control and motor control. Made significant contributions through effective communication and team coordination."
    },
    {
      "organization": "Global Ai Hub",
      "description": "Participated in and managed many bootcamps at the Global AI Hub. Teamed up with many people, managed bootcamps."
    }
  ],
  "skills_and_interests": {
    "technical": ["python", "c", "c#", "java", "SQL", "WEB Programming (Php, Html, Js, Mysql)"],
    "language": "English Professional Work Competence",
    "interests": "Passionate about exploring the latest developments, coding for personal projects, participating in hackathons, staying informed about industry trends. Also follow science fiction and technology-related films. Engaging in outdoor activities like hiking and photography reflects commitment to a balanced lifestyle."
  }
}
"""  # CV JSON formatındaki veriyi buraya yapıştırın

cv_data = json.loads(cv_json)

# BERT modeli ve tokenizer'ını yükle
model_name = "dbmdz/bert-base-turkish-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# CV metnini işle
cv_text = "\n".join([f"{key}: {value}" for key, value in cv_data.items()])
inputs = tokenizer(cv_text, return_tensors="pt", max_length=512, truncation=True)
outputs = model(**inputs)
cv_embedding = outputs.last_hidden_state.mean(dim=1).detach().numpy()

# Startup.json dosyasını yükle
startup_json = """
[
    {
        "title": "Poland",
        "business_name": "PlayerGo",
        "category": "Poland",
        "based_in": "Krakow",
        "tags": "games, hardware, controllers, gaming, videogames",
        "founded": "2010"
    },
    {
        "title": "Turkey",
        "business_name": "TechHub",
        "category": "Technology",
        "based_in": "Istanbul",
        "tags": "tech, startups, innovation, software, hardware",
        "founded": "2015"
    },
    {
        "title": "Germany",
        "business_name": "AutoCraft",
        "category": "Automotive",
        "based_in": "Berlin",
        "tags": "cars, vehicles, automotive, engineering",
        "founded": "2005"
    },
    {
        "title": "Unity Game Development",
        "business_name": "PixelForge Studios",
        "category": "Game Development",
        "based_in": "San Francisco",
        "tags": "unity, game development, virtual reality, augmented reality",
        "founded": "2010"
    }
]
"""  # startup.json formatındaki veriyi buraya yapıştırın

startup_data = json.loads(startup_json)

# startup.json dosyasından şirket metinlerini al
startup_texts = [startup["title"] + " " + startup["business_name"] for startup in startup_data]

# BERT gömme (embedding) işlemlerini yap
startup_embeddings = []
for startup_text in startup_texts:
    inputs = tokenizer(startup_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model(**inputs)
    startup_embedding = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    startup_embeddings.append(startup_embedding)

# Kosinüs benzerliği hesaplama
similarities = [np.dot(cv_embedding, startup_embedding.T) / (np.linalg.norm(cv_embedding) * np.linalg.norm(startup_embedding)) for startup_embedding in startup_embeddings]

# En uygun startup'ı bul
best_match_index = np.argmax(similarities)
best_match_startup = startup_data[best_match_index]

# Sonuçları yazdır
for i, startup in enumerate(startup_data):
    print(f"{startup['title']} {startup['business_name']} ile uyumluluk skoru: {similarities[i]}")

print("En uygun startup:")
print(best_match_startup)


Poland PlayerGo ile uyumluluk skoru: [[0.6565985]]
Turkey TechHub ile uyumluluk skoru: [[0.6231637]]
Germany AutoCraft ile uyumluluk skoru: [[0.65765417]]
Unity Game Development PixelForge Studios ile uyumluluk skoru: [[0.73622453]]
En uygun startup:
{'title': 'Unity Game Development', 'business_name': 'PixelForge Studios', 'category': 'Game Development', 'based_in': 'San Francisco', 'tags': 'unity, game development, virtual reality, augmented reality', 'founded': '2010'}
